In [1]:
import numpy as np
import torch
import h5py

# Using spectral methods to compute the ground truth solution of the PDE
Poisson function is given by:
$-div\ K \nabla u = f$

Helmholtz function is given by:
$- \nabla^2 U + \omega u = f$


Advection-Diffusion function is given by:
$- div\ K\nabla u + v.\nabla u = f$

## Spectral Method with Radial Gaussian Basis Function
Resolution of the grid is $128 \times 128$.\
The given expansion used:\
$$\begin{equation}
f(x) ≈ f^{(N)}(x) = \sum^{N}_{m=1} \phi_i(x)p_i;\ \ \phi_i(x) = \phi(x-x_i)\\
\end{equation}
$$
Here $\phi(x)$ is gaussian with standard diviation $\sigma = \frac{1}{32}$ i.e $4$ times the spatial resolution. Spacing between gaussians is $2\sigma$. $s$ fraction of the $p_i$ value are set to zero and are randomly generated.

We calculate $u$ from $f$ by using by using Fourier Transform of $f$ and then applying the inverse of the required operator and then applying Inverse Fourier Transform to get $u$. To test if our verify the solution, the operator of the equation is applied to $u$ and the norm of it should be very close to $f$

In [ ]:
ntrain = 100 # number of training examples
nval = 25 # number of validation examples
ntest = 25 # number of testing examples

nx = ny = 128 # grid size nxn
lx = ly = 1
dx = lx/nx
dy = ly/ny

std = 1/32
space = 2*std
vfs = [0.2, 0.4, 0.6, 0.8]
ng = 144 # number of gaussians in grid
o1 = 1 # sample wavenumbers starting from o1
o2 = 10 # sample wavenumbers starting from o2
diff_coef_scale = 0.01


x = np.arange(0, lx, dx)
y = np.arange(0, ly, dy)

In [2]:
class Grid:
    def __init__(self, nx=128, ny=128, std=1/32, num_gaussians= 144, Lx=1, Ly=1, isNormalize = True):
        self.nx = nx
        self.ny = ny
        self.Lx = Lx
        self.Ly = Ly
        self.std = std
        self.space = 2 * self.std
        self.dx = self.Lx / self.nx
        self.dy = self.Ly / self.ny
        self.isNormalize = isNormalize
        self.num_gaussians = num_gaussians

        x = np.arange(0, self.Lx, self.dx)
        y = np.arange(0, self.Ly, self.dy)
        self.x_cord, self.y_cord = np.meshgrid(x, y) 
        
    def rbf(self, vf = 0.5, center=(0.5, 0.5), isRemoval_Intregal = True):
        # Implementation of radial basis functions
        num_centers_sqrt = np.sqrt(self.num_gaussians)  # Number of centers in each direction
        grid_length = (num_centers_sqrt - 1) * self.space  # Length of the grid in each direction

        # Generate grid of centers
        centers_x = np.arange(center[0] - grid_length / 2, center[0] + grid_length / 2, self.space)
        centers_y = np.arange(center[1] - grid_length / 2, center[1] + grid_length / 2, self.space)

        # Create meshgrid from centers_x and centers_y
        centers_x_grid, centers_y_grid = np.meshgrid(centers_x, centers_y)

        # Calculate distances using vectorized operations
        grid_distance_squared = ((self.x_cord[..., np.newaxis, np.newaxis] - centers_x_grid)**2 + (self.y_cord[..., np.newaxis, np.newaxis] - centers_y_grid)**2)

        # Generating a random array and zeroing out x percent of its elements
        p = np.where(np.random.rand(*centers_x_grid.shape) < vf, 0, 
                     0.999 * np.random.rand(*centers_x_grid.shape) + 0.001)

        # Reshape p to have a shape compatible with broadcasting with grid
        p_reshaped = p.reshape(1, 1, *p.shape)

        # Calulate the source functions
        R = 2 * self.std**2
        f_basis_form = np.exp(-grid_distance_squared/R)
        f_grid = np.sum(f_basis_form * p_reshaped, axis=(2, 3))

        # Normalize the basis functions
        if self.isNormalize:
            f_grid /= np.max(f_grid)
            
        # Remove integral of the basis function
        if isRemoval_Intregal:
            integral_mean = np.mean(f_grid.flatten())
            f_grid -= integral_mean

        return f_grid

    def fft_coef(self, u, is_torch=False):
        # Function to compute FFT coefficients
        # Get the shape of the input data
        nx, ny = u.shape
        
        # If the input data is not in Torch format
        if not is_torch:
            # Generate the complex wavenumber in x-direction
            ikx = 1j * np.concatenate((np.arange(0, nx // 2 + 1, 1), 
                                       np.arange(-nx // 2 + 1, 0, 1)))
            # Generate the complex wavenumber in y-direction
            iky = 1j * np.concatenate((np.arange(0, ny // 2 + 1, 1), 
                                       np.arange(-ny // 2 + 1, 0, 1)))
            
            # Reshape and repeat to match the shape of the data
            ikx = ikx.reshape(nx, 1).repeat(ny, axis=1)
            iky = iky.reshape(1, ny).repeat(nx, axis=0)
        else:
            # If the input data is in Torch format
            # Generate the complex wavenumber in x-direction
            ikx = 1j * torch.cat((torch.arange(0, nx / 2 + 1, 1), 
                                  torch.arange(-nx / 2 + 1, 0, 1)))
            
            # Generate the complex wavenumber in y-direction
            iky = 1j * torch.cat((torch.arange(0, ny / 2 + 1, 1), 
                                  torch.arange(-ny / 2 + 1, 0, 1)))
            
            # Reshape and repeat using numpy as torch tensor does not support these operations directly
            ikx = np.repeat(ikx.reshape(1, -1), ny, axis=1)
            iky = np.repeat(iky.reshape(-1, 1), nx, axis=0)
        
        return ikx, iky  # Return the computed FFT coefficients
        
    def laplacian(self, u):
        # Function to compute the Laplacian operator of the input function 'u'

        # Computing the Fourier coefficients using helper function fft_coef
        ikx, iky = self.fft_coef(u)

        # Calculating squares of the Fourier coefficients
        ikx2 = ikx ** 2
        iky2 = iky ** 2

        # Performing Fast Fourier Transform (FFT) on input function 'u'
        u_hat = np.fft.fft2(u)

        # Applying the Laplacian operator in frequency domain
        # Multiplying Fourier transformed function by the Laplacian operator in frequency space
        # (ikx^2 + iky^2) * (4 * pi^2) / (Lx * Ly)
        u_hat *= (ikx2 + iky2) * (4.0 * np.pi ** 2) / (self.Lx * self.Ly)

        # Computing the inverse FFT to obtain the final result
        return np.real(np.fft.ifft2(u_hat))


    def grad(self, u):
        # Function to compute the gradient of the input function 'u'

        # Computing the Fourier coefficients using helper function fft_coef
        ikx, iky = self.fft_coef(u)

        # Performing Fast Fourier Transform (FFT) on input function 'u'
        u_hat = np.fft.fft2(u)
        
        # Computing the partial derivative in x-direction using inverse FFT
        ux = np.real(np.fft.ifft2(u_hat * ikx)) * (2.0 * np.pi) / self.Lx
        
        # Computing the partial derivative in y-direction using inverse FFT
        uy = np.real(np.fft.ifft2(u_hat * iky)) * (2.0 * np.pi) / self.Ly
        
        # Returning the computed partial derivatives
        return ux, uy
        
    def div(self, ux, uy):
        # Function to compute the divergence of vector components (ux, uy)
        
        # Getting the Fourier coefficients using the get_fft_coef function
        ikx, iky = self.fft_coef(ux)
        
        # Performing Fast Fourier Transform (FFT) on vector components ux and uy
        ux_hat = np.fft.fft2(ux)
        uy_hat = np.fft.fft2(uy)
    
        # Computing the partial derivative of ux in the x-direction using inverse FFT
        u1 = np.real(np.fft.ifft2(ux_hat * ikx)) * (2.0 * np.pi) / self.Lx
    
        # Computing the partial derivative of uy in the y-direction using inverse FFT
        u2 = np.real(np.fft.ifft2(uy_hat * iky)) * (2.0 * np.pi) / self.Ly
    
        # Returning the sum of computed partial derivatives representing the divergence
        return (u1 + u2)

    def invLaplacian(self, f, offset=0):
        # Function to compute the inverse Laplacian operator of the input function 'f'
        # Supports an offset to avoid division by zero

        # Computing the Fourier coefficients using helper function fft_coef
        ikx, iky = self.fft_coef(f)

        # Calculating squares of the Fourier coefficients
        ikx2 = ikx ** 2
        iky2 = iky ** 2
        
        # Performing Fast Fourier Transform (FFT) on input function 'f'
        f_hat = np.fft.fft2(f)
        
        # Computing the factor for the inverse Laplacian operator with an offset
        factor = ((ikx2 + iky2) * (4.0 * np.pi ** 2) / (self.Lx * self.Ly)) + offset
        
        # Handling potential division by zero by identifying zero values in the factor
        factor_zeros = (factor == 0)

        # Calculating the inverse Laplacian operator on Fourier transformed function 'f_hat'
        # Avoiding division by zero in the factor using np.where for element-wise operations
        u_hat = np.where(factor_zeros, 0, -1 / np.where(factor_zeros, 1, factor)) * np.where(factor_zeros, 0, f_hat)
        
        # Computing the inverse FFT to obtain the final result
        return np.real(np.fft.ifft2(u_hat))

In [3]:
mygrid = Grid()
a = mygrid.rbf()
a

array([[-0.16647863, -0.16647863, -0.16647863, ..., -0.16647863,
        -0.16647863, -0.16647863],
       [-0.16647863, -0.16647863, -0.16647863, ..., -0.16647863,
        -0.16647863, -0.16647863],
       [-0.16647863, -0.16647863, -0.16647863, ..., -0.16647863,
        -0.16647863, -0.16647863],
       ...,
       [-0.16647863, -0.16647863, -0.16647863, ..., -0.16647863,
        -0.16647863, -0.16647863],
       [-0.16647863, -0.16647863, -0.16647863, ..., -0.16647863,
        -0.16647863, -0.16647863],
       [-0.16647863, -0.16647863, -0.16647863, ..., -0.16647863,
        -0.16647863, -0.16647863]])

In [4]:
u_lap = mygrid.laplacian(a)
u_lap

array([[ 6.73658481e-12,  4.38098793e-11,  1.64872317e-10, ...,
        -2.96666836e-12,  1.20141679e-12, -6.36568055e-13],
       [ 1.29227419e-11,  8.55598682e-11,  4.44332050e-10, ...,
         2.43765896e-12,  7.61574723e-13,  5.10209670e-12],
       [ 4.82880437e-11,  2.76472120e-10,  1.36014556e-09, ...,
        -7.10648012e-13, -3.65681509e-12,  1.47186967e-11],
       ...,
       [ 3.70931331e-10,  1.40807280e-09,  4.26092225e-09, ...,
         7.89586490e-12, -2.50147120e-11,  1.33892179e-10],
       [ 7.70252165e-11,  3.07489067e-10,  9.12910046e-10, ...,
         2.13921093e-12, -2.97951263e-12,  3.04901061e-11],
       [ 1.97803386e-11,  5.67945057e-11,  2.11584483e-10, ...,
        -2.86501645e-12, -7.49385173e-13,  5.22010842e-12]])

In [5]:
u_invLap = mygrid.invLaplacian(a)
u_invLap

array([[-0.00484364, -0.00482977, -0.00480996, ..., -0.0048494 ,
        -0.00485345, -0.00485153],
       [-0.00483706, -0.00482276, -0.00480246, ..., -0.00484381,
        -0.00484758, -0.00484533],
       [-0.00482635, -0.0048116 , -0.00479077, ..., -0.00483405,
        -0.00483757, -0.004835  ],
       ...,
       [-0.00483812, -0.00482538, -0.00480676, ..., -0.0048408 ,
        -0.00484584, -0.00484495],
       [-0.0048442 , -0.00483111, -0.00481215, ..., -0.00484791,
        -0.0048526 , -0.00485136],
       [-0.00484603, -0.00483257, -0.0048132 , ..., -0.00485077,
        -0.00485513, -0.00485355]])

In [6]:
u_gradX, u_gradY = mygrid.grad(a)
u_gradX, u_gradY

(array([[ 4.31138121e-15,  2.82902453e-14,  2.58119797e-13, ...,
         -1.21063104e-14, -1.06729895e-14,  4.22933241e-16],
        [ 3.42276980e-14,  2.13338370e-13,  1.06322733e-12, ...,
          1.87098701e-15,  5.65371350e-15, -6.74405511e-15],
        [ 9.98126522e-14,  5.86795091e-13,  3.15231116e-12, ...,
         -5.66431889e-15, -8.80945926e-15,  1.24301741e-15],
        ...,
        [-1.34596741e-12, -4.58346733e-12, -1.45388832e-11, ...,
          1.64385900e-15, -1.93370242e-15, -2.49795978e-15],
        [-2.93228811e-13, -9.63751406e-13, -3.09832570e-12, ...,
         -6.74729599e-15, -5.46455619e-15, -4.74675523e-15],
        [-5.64023674e-14, -1.83980915e-13, -5.44144501e-13, ...,
          1.12298387e-14,  6.76264598e-15,  4.35747114e-15]]),
 array([[ 1.20488464e-14,  8.43655442e-14,  4.29649964e-13, ...,
         -2.02962303e-15, -5.31628188e-15,  9.35196050e-15],
        [ 6.01112603e-14,  2.78221676e-13,  1.43572436e-12, ...,
         -9.16243711e-16, -5.34188101e

In [7]:
u_div = mygrid.div(u_gradX, u_gradY)
u_div

array([[ 6.71408423e-12,  4.31286755e-11,  1.64847150e-10, ...,
        -4.07147593e-12,  1.55523739e-12, -4.24559919e-13],
       [ 1.30093669e-11,  8.62541244e-11,  4.44488087e-10, ...,
         3.64797822e-12,  3.34175340e-13,  5.02515348e-12],
       [ 4.79895364e-11,  2.75484719e-10,  1.35979366e-09, ...,
        -1.92146721e-12, -3.53694427e-12,  1.46803119e-11],
       ...,
       [ 3.73479176e-10,  1.40677066e-09,  4.26359275e-09, ...,
         6.96220065e-12, -2.29969447e-11,  1.31821335e-10],
       [ 7.68183961e-11,  3.06608283e-10,  9.12673880e-10, ...,
         9.36003424e-13, -2.73584387e-12,  3.05792326e-11],
       [ 1.76390893e-11,  5.97464390e-11,  2.09532526e-10, ...,
         6.15546985e-13, -3.34386792e-12,  7.43056440e-12]])

In [8]:
class EquationSolver(Grid):
    def __init__(self, equation_type='helmholtz', nx=128, ny=128, std=1/32, num_gaussians= 144, Lx=1, Ly=1, isNormalize = True, data_path="./"):
        super().__init__(nx, ny, std, num_gaussians, Lx, Ly, isNormalize)
        self.equation_type = equation_type
        self.num_gaussians = num_gaussians
        self.data_path = data_path
    
    def PoissonOperator(self, u, K={'11': 1, '22': 1, '12': 1}):
        # Function to compute Poisson diffusion of the input function 'u'
        # K is a dictionary containing diffusion coefficients

        # Compute gradient of function 'u'
        gradux, graduy = self.grad(u)

        # Compute the components of the diffusion tensor multiplied by gradients
        Kux = K['11'] * gradux + K['12'] * graduy
        Kuy = K['22'] * graduy + K['12'] * gradux
        
        # Compute the divergence of the diffusion tensor multiplied by gradients
        return -self.div(Kux, Kuy)

    def invPoisson(self, f, K={'11': 1, '22': 1, '12': 1}):
        # Function to compute the inverse diffusion operator of the input function 'f'
        # K is a dictionary containing diffusion coefficients

        # Computing the Fourier coefficients using helper function fft_coef
        ikx, iky = self.fft_coef(f)
        
        # Performing Fast Fourier Transform (FFT) on input function 'f'
        f_hat = np.fft.fft2(f)
        
        # Computing the factor for the inverse diffusion operator
        factor = ((K['11'] * (ikx ** 2)) + (K['22'] * (iky ** 2)) + (2 * ikx * iky * K['12'])) * (4.0 * np.pi ** 2) / (
                    self.Lx * self.Ly)
        
        # Handling potential division by zero by identifying zero values in the factor
        factor_zeros = (factor == 0)

        # Calculating the inverse diffusion operator on Fourier transformed function 'f_hat'
        # Avoiding division by zero in the factor using np.where for element-wise operations
        u_hat = np.where(factor_zeros, 0, -1 / np.where(factor_zeros, 1, factor)) * np.where(factor_zeros, 0, f_hat)
        
        # Computing the inverse FFT to obtain the final result in spatial domain
        return np.real(np.fft.ifft2(u_hat))

    def AdvDiffOperator(self, u, K={'11': 1, '22': 1, '12': 1}, v = {'v1': 1, 'v2': 1}):
        # K is a dictionary containing Advection coefficients

        # Compute gradient of function 'u'
        gradux, graduy = self.grad(u)

        # Compute the components of the diffusion tensor multiplied by gradients
        Kux = K['11'] * gradux + K['12'] * graduy
        Kuy = K['22'] * graduy + K['12'] * gradux

        # Compute the components of the Advection tensor multiplied by gradients
        diffusion_part = -self.div(Kux, Kuy)
        advection_part = (v['v1']*gradux + v['v2']*graduy)

        # Computing the ratio of the diffusion part and advection part
        ratio = np.linalg.norm(diffusion_part)/np.linalg.norm(advection_part)
        
        # Compute the divergence of the diffusion tensor multiplied by gradients
        return diffusion_part + advection_part, ratio
        

    def invAdvDiff(self, f, K={'11': 1, '22': 1, '12': 1}, v = {'v1': 1, 'v2': 1}):
        # Function to compute the inverse diffusion operator of the input function 'f'
        # K is a dictionary containing diffusion coefficients

        # Computing the Fourier coefficients using helper function fft_coef
        ikx, iky = self.fft_coef(f)
        
        # Performing Fast Fourier Transform (FFT) on input function 'f'
        f_hat = np.fft.fft2(f)
        
        # Computing the factor for the inverse diffusion part of the operator
        diff_factor = ((K['11'] * (ikx ** 2)) + (K['22'] * (iky ** 2)) + (2 * ikx * iky * K['12'])) * (4.0 * np.pi ** 2) / (
                        self.Lx * self.Ly)
        
        # Computing the factor for the inverse advection part of the operator
        advection_factor = (v['v1']*ikx + v['v2']*iky)*(2.0 * np.pi) / (self.Lx)

        # Computing the factor for the inverse advection operator
        factor = diff_factor - advection_factor

        # Computing the ratio of the diffusion part and advection part
        ratio = np.linalg.norm(diff_factor)/np.linalg.norm(advection_factor)
        
        # Handling potential division by zero by identifying zero values in the factor
        factor_zeros = (factor == 0)

        # Calculating the inverse diffusion operator on Fourier transformed function 'f_hat'
        # Avoiding division by zero in the factor using np.where for element-wise operations
        u_hat = np.where(factor_zeros, 0, -1 / np.where(factor_zeros, 1, factor)) * np.where(factor_zeros, 0, f_hat)
        
        # Computing the inverse FFT to obtain the final result in spatial domain
        return np.real(np.fft.ifft2(u_hat))
        
    def helmholtz(self, omega_lower_limit = 0, omega_upper_limit = 1):
        # Function to solve the Helmholtz equation
        
        # Initializing error value to a high number
        error = 100
        
        # Iterate until the error is below the threshold
        while error > 1E-5:
            # Generate a source using radial basis functions
            source = self.rbf(isRemoval_Intregal = False)
            
            # Randomly select omega within specified limits
            omega = np.random.randint(omega_lower_limit, omega_upper_limit + 1)
            
            # Solve the Helmholtz equation with the inverse Laplacian operator
            u_self = self.invLaplacian(source, omega)
            
            # Compute the left-hand side of the Helmholtz equation
            lhs = self.laplacian(u_self) + omega * u_self
            
            # Compute the error based on the norm differences between lhs and source
            error = np.abs(np.linalg.norm(lhs) - np.linalg.norm(source))
            
            # Check convergence and print error if it doesn't converge
            if error > 1E-5:
                print("Did not converge. Error at:", error)
        
        # Return the solution 'u_self', the source, and the chosen omega
        return u_self, source, omega
        
    def poisson(self, diffusion_eigenvalues_lower = 1, diffusion_eigenvalues_upper = 5):
        # Function to solve the Poisson equation 
        
        # Initializing error value to a high number
        error = 100
        
        # Iterate until the error is below the threshold
        while error > 1E-5:
            # Generate a source using radial basis functions
            source = self.rbf()
            
            # Randomly create a rotation matrix
            theta_rot = np.random.rand() * 2 * np.pi
            rot = np.array([[np.cos(theta_rot), -np.sin(theta_rot)], [np.sin(theta_rot), np.cos(theta_rot)]])
            
            # Create a random symmetric positive definite matrix
            A = np.array([[1, 0], [0, diffusion_eigenvalues_upper + np.random.rand() * (diffusion_eigenvalues_upper - diffusion_eigenvalues_lower)]])
            
            # Compute the similarity transformation matrix K_matrix to preserve eigenvalues
            K_matrix = np.linalg.inv(rot) @ (A @ rot)
            k = {'11': K_matrix[0, 0], '22': K_matrix[1, 1], '12': K_matrix[0, 1]}
            
            # Solve the Poisson equation with the inverse Poisson operator
            u_self = self.invPoisson(source, K=k)

            # Compute the left-hand side of the Poisson equation
            lhs = self.PoissonOperator(u_self, k)
            
            # Compute the error based on the norm differences between lhs and source
            error = np.abs(np.linalg.norm(lhs) - np.linalg.norm(source))
            
            # Check convergence and print error if it doesn't converge
            if error > 1E-5:
                print("Did not converge. Error at:", error)
            
        # Return the solution 'u_self', the source, and the constructed 'k' matrix
        return u_self, source, k

    def advection_diffusion(self, diffusion_eigenvalues_lower = 1, diffusion_eigenvalues_upper = 5):
        # Implementation for solving Advection-Diffusion equation using Grid methods

        # Initializing error value to a high number
        error = 100
        
        # Iterate until the error is below the threshold
        while error > 1E-5:
            # Generate a source using radial basis functions
            source = self.rbf()
            
            # Randomly create a rotation matrix
            theta_rot = np.random.rand() * 2 * np.pi
            rot = np.array([[np.cos(theta_rot), -np.sin(theta_rot)], [np.sin(theta_rot), np.cos(theta_rot)]])
            
            # Create a random symmetric positive definite matrix
            A = np.array([[1, 0], [0, diffusion_eigenvalues_upper + np.random.rand() * (diffusion_eigenvalues_upper - diffusion_eigenvalues_lower)]])
            
            # Compute the similarity transformation matrix K_matrix to preserve eigenvalues
            K_matrix = np.linalg.inv(rot) @ (A @ rot)
            k = {'11': K_matrix[0, 0], '22': K_matrix[1, 1], '12': K_matrix[0, 1]}
            
            theta_velocity = np.random.rand() * 2 * np.pi
            r = 1
            v = {'v1': r*np.cos(theta_velocity), 'v2':r*np.sin(theta_velocity)}
            
            # Solve the Poisson equation with the inverse Poisson operator
            u_self = self.invAdvDiff(source, K=k, v=v)

            # Compute the left-hand side of the Poisson equation
            lhs, ratio = self.AdvDiffOperator(u_self, K=k, v=v)
            
            # Compute the error based on the norm differences between lhs and source
            error = np.abs(np.linalg.norm(lhs) - np.linalg.norm(source))
            
            # Check convergence and print error if it doesn't converge
            if error > 1E-5:
                print("Did not converge. Error at:", error)

        return u_self, source, k, v, ratio
        
        # Open the HDF5 file in append mode
        with h5py.File(path, "a") as file:
            try:
                # Delete existing 'fields' and 'tensor' datasets if they already exist
                del file['fields']
                del file['tensor']
            except KeyError:
                # Ignore KeyError if datasets do not exist
                pass

            # Create 'fields' dataset and store data array
            file.create_dataset('fields', data_array.shape, dtype='<f4', data=data_array)

            # Create 'tensor' dataset and store tensor array
            file.create_dataset('tensor', tensor_array.shape, dtype='<f4', data=tensor_array)

    def generate_data(self, ntrain=100, nval=25, ntest=25, e1=1, e2=5, o1=1, o2=10):
        train = np.zeros((ntrain, 2, self.grid_size, self.grid_size))
        train_info = np.zeros((ntrain, 2))
        val = np.zeros((nval, 2, self.grid_size, self.grid_size))
        val_info = np.zeros((nval, 2))
        test = np.zeros((ntest, 2, self.grid_size, self.grid_size))
        test_info = np.zeros((ntest, 2))

        for idx, vf in enumerate([0.2, 0.4, 0.6, 0.8]):
            if self.equation_type == 'helmholtz':
                u, source, info = self.helmholtz()
            elif self.equation_type == 'poisson':
                u, source, info = self.poisson()
            elif self.equation_type == 'advdiff':
                u, source, info = self.advection_diffusion()

            # Assign data to appropriate datasets (train, validation, test)

        # Save data into HDF5 files


In [9]:
helmholtz_eq = EquationSolver(equation_type='poisson')
helmholtz_sol, helmholtz_f, helmholtz_omega = helmholtz_eq.poisson()
print("Helmholtz, u:", helmholtz_sol)
print("f:", helmholtz_f)
print("omega:", helmholtz_omega)

Helmholtz, u: [[-0.00262542 -0.00259005 -0.00254798 ... -0.00269108 -0.00267596
  -0.00265407]
 [-0.00262444 -0.00258876 -0.00254638 ... -0.00269088 -0.00267552
  -0.00265336]
 [-0.00262338 -0.00258741 -0.00254471 ... -0.00269061 -0.002675
  -0.00265258]
 ...
 [-0.00262795 -0.00259344 -0.00255229 ... -0.00269119 -0.00267685
  -0.00265576]
 [-0.00262718 -0.00259239 -0.00255094 ... -0.00269123 -0.00267663
  -0.00265527]
 [-0.00262634 -0.00259126 -0.0025495  ... -0.0026912  -0.00267633
  -0.00265471]]
f: [[-0.15926034 -0.15926034 -0.15926034 ... -0.15926034 -0.15926034
  -0.15926034]
 [-0.15926034 -0.15926034 -0.15926034 ... -0.15926034 -0.15926034
  -0.15926034]
 [-0.15926034 -0.15926034 -0.15926034 ... -0.15926034 -0.15926034
  -0.15926034]
 ...
 [-0.15926034 -0.15926034 -0.15926034 ... -0.15926034 -0.15926034
  -0.15926034]
 [-0.15926034 -0.15926034 -0.15926034 ... -0.15926034 -0.15926034
  -0.15926034]
 [-0.15926034 -0.15926034 -0.15926034 ... -0.15926034 -0.15926034
  -0.15926034]]
o

In [10]:
poisson_eq = EquationSolver(equation_type='poisson')
poisson_sol, poisson_f, poisson_k = poisson_eq.poisson()

print("Poisson, u:", poisson_sol)
print("f:", poisson_f)
print("k:", poisson_k)

Poisson, u: [[-0.00215667 -0.00210731 -0.00205193 ... -0.00226824 -0.00223717
  -0.00219997]
 [-0.00215573 -0.00210642 -0.00205103 ... -0.00226685 -0.00223598
  -0.00219893]
 [-0.00215464 -0.00210535 -0.00204993 ... -0.00226535 -0.00223467
  -0.00219776]
 ...
 [-0.00215854 -0.00210889 -0.00205337 ... -0.00227176 -0.00224001
  -0.00220227]
 [-0.00215808 -0.00210854 -0.0020531  ... -0.0022707  -0.00223919
  -0.00220164]
 [-0.00215745 -0.00210801 -0.00205262 ... -0.00226953 -0.00223824
  -0.00220088]]
f: [[-0.14633863 -0.14633863 -0.14633863 ... -0.14633863 -0.14633863
  -0.14633863]
 [-0.14633863 -0.14633863 -0.14633863 ... -0.14633863 -0.14633863
  -0.14633863]
 [-0.14633863 -0.14633863 -0.14633862 ... -0.14633863 -0.14633863
  -0.14633863]
 ...
 [-0.14633863 -0.14633863 -0.14633863 ... -0.14633863 -0.14633863
  -0.14633863]
 [-0.14633863 -0.14633863 -0.14633863 ... -0.14633863 -0.14633863
  -0.14633863]
 [-0.14633863 -0.14633863 -0.14633863 ... -0.14633863 -0.14633863
  -0.14633863]]
k

In [11]:
advdiff_eq = EquationSolver(equation_type='advdiff')
advdiff_sol, advdiff_f, advdiff_k, advdiff_v, advdiff_ratio = advdiff_eq.advection_diffusion()

print("Steady State Advection Diffusion, u:", advdiff_sol)
print("f:", advdiff_f)
print("k:", advdiff_k)
print("v:", advdiff_v)
print("ratio:", advdiff_ratio)

Steady State Advection Diffusion, u: [[-0.00151952 -0.0015134  -0.00150597 ... -0.00153015 -0.00152787
  -0.00152433]
 [-0.00150503 -0.00149888 -0.00149136 ... -0.00151553 -0.00151333
  -0.00150984]
 [-0.00148889 -0.00148267 -0.00147504 ... -0.00149936 -0.00149721
  -0.00149373]
 ...
 [-0.00155324 -0.00154702 -0.00153959 ... -0.00156486 -0.00156215
  -0.00155828]
 [-0.00154361 -0.00153746 -0.00153007 ... -0.00155481 -0.00155227
  -0.00154854]
 [-0.00153238 -0.00152626 -0.00151887 ... -0.00154324 -0.00154085
  -0.00153723]]
f: [[-0.16808191 -0.16808191 -0.16808191 ... -0.16808191 -0.16808191
  -0.16808191]
 [-0.16808191 -0.16808191 -0.16808191 ... -0.16808191 -0.16808191
  -0.16808191]
 [-0.16808191 -0.16808191 -0.16808191 ... -0.16808191 -0.16808191
  -0.16808191]
 ...
 [-0.16808191 -0.16808191 -0.16808191 ... -0.16808191 -0.16808191
  -0.16808191]
 [-0.16808191 -0.16808191 -0.16808191 ... -0.16808191 -0.16808191
  -0.16808191]
 [-0.16808191 -0.16808191 -0.16808191 ... -0.16808191 -0.1